In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import datetime
import time
import matplotlib
import base
import predict

In [2]:
#读取user_pay_count
user_pay_counts = base.ReadUserPayCountCSV();
#读取开店时间数据
shop_open_dates = base.ReadShopOpenDateCSV();
###读取商家信息
shop_infos = base.ReadShopInfoCSV();
##读取城市名称对照表
city_names = base.ReadCityNameCSV();
#读取休假表
calenders = base.ReadCalendarCSV();

In [3]:
holiday_mean_df = predict.trainHolidayMeansData(
    shop_open_dates, user_pay_counts, calenders, pd.to_datetime('2016-9-27'), pd.to_datetime('2016-10-17'))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [4]:
start_time = pd.to_datetime('10/18/2016');
end_time = pd.to_datetime('10/31/2016');
predict_date = pd.date_range(start=start_time, end=end_time, freq='D', normalize=True)
pridicts = {}
evaluations = {}
for id in range(1, 2001):
    y_Pred = []
    for date in predict_date:
        daytype = calenders.loc[date.strftime('%Y-%m-%d')]['daytype']
        y_Pred.append(holiday_mean_df.loc[id][daytype]);
    pridicts[id] = y_Pred
    y_Test = base.getTestCount(user_pay_counts, id, start_time, end_time).values
    e = base.evaluation(y_Test, y_Pred)
    print("id=%d evaluation=%f"%(id, e))
    pridicts[id] = y_Pred
    evaluations[id] = e

id=1 evaluation=0.068039
id=2 evaluation=0.529337
id=3 evaluation=0.070383
id=4 evaluation=0.115067
id=5 evaluation=0.245292
id=6 evaluation=0.084917
id=7 evaluation=0.068928
id=8 evaluation=0.111545
id=9 evaluation=0.048330
id=10 evaluation=0.105626
id=11 evaluation=0.088142
id=12 evaluation=0.166974
id=13 evaluation=0.034992
id=14 evaluation=0.092993
id=15 evaluation=0.044327
id=16 evaluation=0.138709
id=17 evaluation=0.032091
id=18 evaluation=0.240847
id=19 evaluation=0.062358
id=20 evaluation=0.033599
id=21 evaluation=0.092688
id=22 evaluation=0.106106
id=23 evaluation=0.354333
id=24 evaluation=0.169486
id=25 evaluation=0.038431
id=26 evaluation=0.053277
id=27 evaluation=0.085501
id=28 evaluation=0.103667
id=29 evaluation=0.079991
id=30 evaluation=0.070272
id=31 evaluation=0.087384
id=32 evaluation=0.076402
id=33 evaluation=0.080004
id=34 evaluation=0.033576
id=35 evaluation=0.119012
id=36 evaluation=0.083460
id=37 evaluation=0.055281
id=38 evaluation=0.082241
id=39 evaluation=0.13

In [5]:
evaluation_df = pd.DataFrame.from_dict(evaluations, orient='index');
evaluation_df.columns = ['evaluation']
evaluation_df.index.name = 'id'
evaluation_df.sort_values('evaluation')


,evaluation
id,
1624,0.022499
900,0.023025
1876,0.024826
516,0.025474
220,0.025493
1922,0.025989
225,0.026567
1365,0.026973
100,0.027099


In [12]:
evaluation_df = evaluation_df.fillna(1)
evaluation_df.to_csv("./data/dataset/dataset/validation/holiday_mean_evaluation.txt", sep='\t', index=True, encoding='UTF-8')

In [13]:
evaluation_df['evaluation'].mean()

0.09716518739517316

In [6]:
def initHolidayMeanData(id):
    open_time = shop_open_dates.loc[id]['date'];
    if open_time < pd.to_datetime('2016-9-27'):
        open_time = pd.to_datetime('2016-9-27')
    df = base.countShopPayTimePeriods(user_pay_counts, id, date_range=[open_time, pd.to_datetime('2016-10-17')], time_range=[datetime.timedelta(hours=0), datetime.timedelta(hours=23)])
#     df = series.to_frame()
#     df = df.rename(columns = {0:'count'})
    df['holiday'] = calenders.loc[df.index.strftime('%Y-%m-%d')]['daytype'].values
    
    mean_arr = []
    std_arr = []
    mean_dict = {}
    for i in range(1, 6):
        temp_df = df[df['holiday'] == i];
        ###去除部分异常值
        #count_max = temp_df['count'].quantile(0.98)
        #count_min = temp_df['count'].quantile(0.02)
        #print(temp_df)
        #valid_df = temp_df[(temp_df['count'] <= count_max) & (temp_df['count'] >= count_min)]
        #print(valid_df)
        mean_dict[i] = [temp_df['count'].mean(), temp_df['count'].std()]
        
    #print(mean_dict)
    result_df = pd.DataFrame(mean_dict).T
    result_df.rename(columns={0: 'mean', 1: 'std'}, inplace=True) 

    return result_df

In [9]:
initHolidayMeanData(700)

,mean,std
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0


In [10]:
base.getTestCount(user_pay_counts, 700, start_time, end_time).values

array([  0,   0,   0,   0,   0,  33,  62,  70,  65,  80,  91, 114,  77,  67], dtype=int64)

In [38]:
holiday_mean_df = pd.DataFrame()
for id in range(1,2001):
    print(id)
    df = initHolidayMeanData(id)
    holiday_mean_df[id] = df['mean'].T

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [39]:
holiday_mean_df = holiday_mean_df.T
holiday_mean_df.index.name = 'id'
holiday_mean_df = holiday_mean_df.astype(np.int)

,1,2,3,4,5
id,,,,,
1,221.2500,262.666667,281.666667,251.8,260.00
2,43.8750,61.000000,85.666667,109.2,85.50
3,67.7500,100.666667,125.666667,103.2,92.25
4,94.6875,117.000000,176.333333,113.8,169.00
5,240.7500,151.333333,163.666667,38.6,146.75
6,77.3125,91.333333,86.000000,58.8,91.50
7,113.8125,140.333333,202.666667,185.2,168.25
8,63.7500,68.333333,90.666667,73.6,73.50
9,223.0625,237.000000,267.000000,215.2,290.75


In [40]:
def getTestCount(id):
    df = base.countShopPayTimePeriods(user_pay_counts, id, date_range=[pd.to_datetime('2016-10-18'), pd.to_datetime('2016-10-31')], 
                                     time_range=[datetime.timedelta(hours=0), datetime.timedelta(hours=23)])
    return df['count'];

In [50]:
predict_date = pd.date_range(start='10/18/2016', end='10/31/2016', freq='D', normalize=True)
pridicts = {}
evaluations = {}
for id in range(1, 2001):
    y_Pred = []
    for date in predict_date:
        daytype = calenders.loc[date.strftime('%Y-%m-%d')]['daytype']
        y_Pred.append(holiday_mean_df.loc[id][daytype]);
    pridicts[id] = y_Pred
    y_Test = getTestCount(id).values
    e = base.evaluation(y_Test, y_Pred)
    print("id=%d evaluation=%f"%(id, e))
    pridicts[id] = y_Pred
    evaluations[id] = e

id=1 evaluation=0.062925
id=2 evaluation=0.398929
id=3 evaluation=0.064159
id=4 evaluation=0.104484
id=5 evaluation=0.256073
id=6 evaluation=0.117743
id=7 evaluation=0.061468
id=8 evaluation=0.105710
id=9 evaluation=0.048016
id=10 evaluation=0.109385
id=11 evaluation=0.091218
id=12 evaluation=0.170525
id=13 evaluation=0.031441
id=14 evaluation=0.119198
id=15 evaluation=0.044882
id=16 evaluation=0.128028
id=17 evaluation=0.040691
id=18 evaluation=0.232957
id=19 evaluation=0.064969
id=20 evaluation=0.048218
id=21 evaluation=0.059170
id=22 evaluation=0.119919
id=23 evaluation=0.538217
id=24 evaluation=0.161726
id=25 evaluation=0.041388
id=26 evaluation=0.056901
id=27 evaluation=0.084810
id=28 evaluation=0.274310
id=29 evaluation=0.074242
id=30 evaluation=0.077414
id=31 evaluation=0.081297
id=32 evaluation=0.066646
id=33 evaluation=0.062338
id=34 evaluation=0.040708
id=35 evaluation=0.116368
id=36 evaluation=0.073553
id=37 evaluation=0.052267
id=38 evaluation=0.074381
id=39 evaluation=0.14

In [66]:
evaluation_df = pd.DataFrame.from_dict(evaluations, orient='index');
evaluation_df.columns = ['evaluation']
evaluation_df.index.name = 'id'
evaluation_df.sort_values('evaluation')
evaluation_df = evaluation_df.fillna(1)
evaluation_df.to_csv("./data/dataset/dataset/holiday_mean_evaluation.txt", sep='\t', index=True, encoding='UTF-8')

In [67]:
evaluation_df[evaluation_df['evaluation'].isnull()]

,evaluation
id,


In [ ]:
predict_df.to_csv("./data/dataset/dataset/holiday_mean_prediction.txt", sep='\t', header = None, index=True, encoding='UTF-8')

In [53]:
predict_df = pd.DataFrame(pridicts)
predict_df = predict_df.T
predict_df.to_csv("./data/dataset/dataset/holiday_mean_prediction.txt", sep='\t', header = None, index=True, encoding='UTF-8')